# Transforming British Columbia’s Tuberculosis Data to a Common Format
 
## Overview
 
Provinces and territories send the TB team two kinds of files: Cases and Outcomes. These files should be uploaded to Google Cloud Storage . Once uploaded, the data in the files should be transformed to a common format using the steps described below. 
 
Transformation of this data is done using Python, along with the package [pandas](https://pandas.pydata.org) and its integration with [DuckDB's Python API](https://duckdb.org/docs/api/python/overview.html).
 
Pandas allows us to read data into tabular structure, called DataFrames and DuckDB will allow us to execute data manipulations and transformations on the DataFrames.
 
We start by importing necessary modules used throughout the notebook. It’s  standard practice to import all the modules in the first cell of the notebook. This improves readability, and  and keeps the notebook organized.

### Step 0. Install required packages

Vertex AI provides most packages used in the notebook. Packages can be added to the environment of the notebook while creation or can be manually installed inline through terminal commands on jupyter cells.


In [1554]:
%pip install duckdb
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


We can install more packages inline this way - the `%` is used to indicate, the above is a magic command, something that lets us execute additional functionality than standard python/R code.

### Step 1. Importing necessary modules
 
Pandas and duckdb are imported to help with the cleaning and transformations:.

In [1555]:
import pandas as pd
from pathlib import Path
import duckdb

The file path is constructed using the module pathlib. Path(https://docs.python.org/3/library/pathlib.html). It allows to create, manipulate and access file paths in a platform independent manner, For instance OS specific file path delimiters like "test\file" for Windows, our local systems and "test/file" for an unix based OS, like the most running jupyter server on a cloud environment. It also helps keep the code maintainable and readable for operations like to check existance of files and directories and to join the paths.
 
Reading the file is done using Pandas. Pandas lets you read excel files, through an additional plugin and store them as a [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)

In [1556]:
DATA_FILE=Path(".", "tests", "BCCaseFileUploading.xlsx")
df = pd.read_excel(DATA_FILE)

## Case Files

### Step 2.1 Writing transformation queries
The first step in standardizing the PT data to a uniform format involves several transformations. These include renaming fields with suitable terms, creating necessary variables for comprehensive reporting based on existing fields, and reconfiguring specific string values into numeric classifications. 

For easing into using Python for the data processing, we recommend using DuckDB's Python API can be used to run SQL queries on the loaded dataframes. Data transformations and manipulation can be performed by writing SQL queries and executing them on dataframes using the `duckdb.sql` method. 



We divide the transformation process into some sub-steps for ease of debugging. 

#### Step 2.1.1. Writing a query for renamed fields

In [1557]:
SQL_QUERY = """SELECT
    client_id_phac AS RegisterCaseNumber,
    classification_date AS DiagnosisDate,
    classification_year AS Survey_Year,
    '' AS IR_Gatifloxacin,
    '' AS IR_Levofloxacin ,
    '' AS IR_Ciprofloxacin ,
    '' AS IR_Clarithromycin,
    '' AS IR_Clofazamine ,
    '' AS IR_Cycloserine ,
    '' AS IR_Pyridoxine ,
    '' AS IR_Rifapentine ,
    '' AS IR_Unknown ,
    age_at_classification_date_years AS Age,
    ROW_NUMBER() OVER (ORDER BY client_id_phac) AS rownumber
FROM df
"""

We now pass the above query to the `duckdb.sql` method that returns a DuckDBPyRelation object. This object contains the `to_df` method that converts the DuckDBPyRelation object back to a pandas DataFrame, which can be used for further manipulations.

In [1558]:
# This line and the 2 lines below are commented. Removing the '#' runs the code. The line below this description executes the SQL query on 
# top of the dataframe and gives us the result
# result = duckdb.sql(SQL_QUERY).to_df()

result = pd.DataFrame({
    'RegisterCaseNumber': df['client_id_phac'],
    'DiagnosisDate': df['classification_date'],
    'Survey_Year': df['classification_year'],
    'IR_Gatifloxacin': '',
    'IR_Levofloxacin': '',
    'IR_Ciprofloxacin': '',
    'IR_Clarithromycin': '',
    'IR_Clofazamine': '',
    'IR_Cycloserine': '',
    'IR_Pyridoxine': '',
    'IR_Rifapentine': '',
    'IR_Unknown': '',
    'Age': df['age_at_classification_date_years'],
    'rownumber': range(1, len(df) + 1)  # Just like ROW_NUMBER()
})

result

,RegisterCaseNumber,DiagnosisDate,Survey_Year,IR_Gatifloxacin,IR_Levofloxacin,IR_Ciprofloxacin,IR_Clarithromycin,IR_Clofazamine,IR_Cycloserine,IR_Pyridoxine,IR_Rifapentine,IR_Unknown,Age,rownumber
0,1,2013-07-10,2013,,,,,,,,,,61,1
1,2,2021-07-10,2021,,,,,,,,,,73,2
2,3,2017-07-25,2017,,,,,,,,,,22,3
3,4,2019-04-10,2019,,,,,,,,,,33,4
4,5,2021-12-20,2021,,,,,,,,,,55,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,2017-04-26,2017,,,,,,,,,,24,996
996,997,2022-09-22,2022,,,,,,,,,,44,997
997,998,2017-12-28,2017,,,,,,,,,,35,998
998,999,2021-04-30,2021,,,,,,,,,,49,999


#### Step 2.1.2. Add derived columns to the result Dataframe

##### CaseCriteria and InitialResistance

In [1559]:
SQL_QUERY = """
SELECT
    CASE
        WHEN lower(classification) = 'clinical' THEN 2
        WHEN lower(classification) = 'confirmed' THEN 1
    END AS CaseCriteria,
    CASE
        WHEN classification = 'Confirmed' AND
            (isoniazid_resistance LIKE 'R'
            OR ethambutol_resistance LIKE 'R'
            OR rifampin_resistance LIKE 'R'
            OR pyrazinamide_resistance LIKE 'R') THEN 1
        WHEN classification = 'Confirmed' AND
            (isoniazid_resistance LIKE 'S'
            OR ethambutol_resistance LIKE 'S'
            OR rifampin_resistance LIKE 'S'
            OR pyrazinamide_resistance LIKE 'S') THEN 2
        WHEN classification = 'Confirmed' AND
            (isoniazid_resistance IS NULL
            AND ethambutol_resistance IS NULL
            AND rifampin_resistance IS NULL
            AND pyrazinamide_resistance IS NULL) THEN 9
    END AS Initial_Resistance
from df
"""
# sub_result = duckdb.sql(SQL_QUERY).to_df()

The 2 dataframes are of equal length and we can combine the new fields to the result by simply concatenating them.

In [1560]:
# result = pd.concat([result, sub_result], axis=1)

In [1561]:
# We don't require the above concatenations to add a field directly using pandas methods instead of the SQL.

result["CaseCriteria"] = df["classification"].map({
    "Clinical": 2,
    "Confirmed": 1
})

def calculate_initial_resistance(row):
    if row['classification'] == 'Confirmed':
        if 'R' in [row['isoniazid_resistance'], row['ethambutol_resistance'], row['rifampin_resistance'], row['pyrazinamide_resistance']]:
            return 1
        elif 'S' in [row['isoniazid_resistance'], row['ethambutol_resistance'], row['rifampin_resistance'], row['pyrazinamide_resistance']]:
            return 2
        elif pd.isnull(row['isoniazid_resistance']) and pd.isnull(row['ethambutol_resistance']) and pd.isnull(row['rifampin_resistance']) and pd.isnull(row['pyrazinamide_resistance']):
            return 9
    return None

result['Initial_Resistance'] = df.apply(calculate_initial_resistance, axis=1)

result.head(10)

,RegisterCaseNumber,DiagnosisDate,Survey_Year,IR_Gatifloxacin,IR_Levofloxacin,IR_Ciprofloxacin,IR_Clarithromycin,IR_Clofazamine,IR_Cycloserine,IR_Pyridoxine,IR_Rifapentine,IR_Unknown,Age,rownumber,CaseCriteria,Initial_Resistance
0,1,2013-07-10,2013,,,,,,,,,,61,1,2,NaN
1,2,2021-07-10,2021,,,,,,,,,,73,2,1,1.0
2,3,2017-07-25,2017,,,,,,,,,,22,3,2,NaN
3,4,2019-04-10,2019,,,,,,,,,,33,4,1,1.0
4,5,2021-12-20,2021,,,,,,,,,,55,5,1,2.0
5,6,2015-02-27,2015,,,,,,,,,,65,6,1,1.0
6,7,2020-10-28,2020,,,,,,,,,,70,7,2,NaN
7,8,2014-02-27,2014,,,,,,,,,,72,8,2,NaN
8,9,2017-11-28,2017,,,,,,,,,,24,9,2,NaN
9,10,2016-11-05,2016,,,,,,,,,,37,10,2,NaN


##### Drug Resistance fields

In [1562]:
SQL_QUERY = """
SELECT
    CASE
        WHEN isoniazid_resistance LIKE 'R' THEN 'Resistant'
        WHEN isoniazid_resistance LIKE 'S' THEN 'Susceptible'
        ELSE 'Not Done'
    END AS IR_INH,
    CASE
        WHEN ethambutol_resistance LIKE 'R' THEN 'Resistant'
        WHEN ethambutol_resistance LIKE 'S' THEN 'Susceptible'
        ELSE 'Not Done'
    END AS IR_EMB,
    CASE
        WHEN rifampin_resistance LIKE 'R' THEN 'Resistant'
        WHEN rifampin_resistance LIKE 'S' THEN 'Susceptible'
        ELSE 'Not Done'
    END AS IR_RMP,
    CASE
        WHEN pyrazinamide_resistance LIKE 'R' THEN 'Resistant'
        WHEN pyrazinamide_resistance LIKE 'S' THEN 'Susceptible'
        ELSE 'Not Done'
    END AS IR_PZA,
    CASE
        WHEN amikacin_resistance LIKE 'R' THEN 'Resistant'
        WHEN amikacin_resistance LIKE 'S' THEN 'Susceptible'
        ELSE 'Not Done'
    END AS IR_Amikacin,
    CASE
        WHEN capreomycin_resistance LIKE 'R' THEN 'Resistant'
        WHEN capreomycin_resistance LIKE 'S' THEN 'Susceptible'
        ELSE 'Not Done'
    END AS IR_Capreomycin,
    CASE
        WHEN ethionamide_resistance LIKE 'R' THEN 'Resistant'
        WHEN ethionamide_resistance LIKE 'S' THEN 'Susceptible'
        ELSE 'Not Done'
    END AS IR_Ethionamide,
    CASE
        WHEN kanamycin_resistance LIKE 'R' THEN 'Resistant'
        WHEN kanamycin_resistance LIKE 'S' THEN 'Susceptible'
        ELSE 'Not Done'
    END AS IR_Kanamycin,
    CASE
        WHEN moxifloxacin_resistance LIKE 'R' THEN 'Resistant'
        WHEN moxifloxacin_resistance LIKE 'S' THEN 'Susceptible'
        ELSE 'Not Done'
    END AS IR_Moxifloxacin,
    CASE
        WHEN ofloxacin_resistance LIKE 'R' THEN 'Resistant'
        WHEN ofloxacin_resistance LIKE 'S' THEN 'Susceptible'
        ELSE 'Not Done'
    END AS IR_Ofloxacin,
    CASE
        WHEN linezolid_resistance LIKE 'R' THEN 'Resistant'
        WHEN linezolid_resistance LIKE 'S' THEN 'Susceptible'
        ELSE 'Not Done'
    END AS IR_Other,
    CASE
        WHEN linezolid_resistance IS NOT NULL THEN 'Linezolid'
    END AS IR_Other_Drug,
    CASE
        WHEN para_aminosalicylate_resistance LIKE 'R' THEN 'Resistant'
        WHEN para_aminosalicylate_resistance LIKE 'S' THEN 'Susceptible'
        ELSE 'Not Done'
    END AS IR_PAS,
    CASE
        WHEN rifabutin_resistance LIKE 'R' THEN 'Resistant'
        WHEN rifabutin_resistance LIKE 'S' THEN 'Susceptible'
        ELSE 'Not Done'
    END AS IR_Rifabutin,
    CASE
        WHEN streptomycin_resistance LIKE 'R' THEN 'Resistant'
        WHEN streptomycin_resistance LIKE 'S' THEN 'Susceptible'
        ELSE 'Not Done'
    END AS IR_SM
from df
"""
# sub_result = duckdb.sql(SQL_QUERY).to_df()
# result = pd.concat([result, sub_result], axis=1)

In [1563]:
# Again, no need to concatenate when using pandas methods

# All conversions seem to be similar so we can use one method
def map_resistance(value):
    if value == 'R':
        return 'Resistant'
    elif value == 'S':
        return 'Susceptible'
    else:
        return 'Not Done'

result['IR_INH'] = df['isoniazid_resistance'].apply(map_resistance)
result['IR_EMB'] = df['ethambutol_resistance'].apply(map_resistance)
result['IR_RMP'] = df['rifampin_resistance'].apply(map_resistance)
result['IR_PZA'] = df['pyrazinamide_resistance'].apply(map_resistance)
result['IR_AMIKACIN'] = df['amikacin_resistance'].apply(map_resistance)
result['IR_CAPREOMYCIN'] = df['capreomycin_resistance'].apply(map_resistance)
result['IR_ETHIONAMIDE'] = df['ethionamide_resistance'].apply(map_resistance)
result['IR_KANAMYCIN'] = df['kanamycin_resistance'].apply(map_resistance)
result['IR_MOXIFLOXACIN'] = df['moxifloxacin_resistance'].apply(map_resistance)
result['IR_OFLOXACIN'] = df['ofloxacin_resistance'].apply(map_resistance)
result['IR_OTHER'] = df['linezolid_resistance'].apply(map_resistance)
result['IR_PAS'] = df['para_aminosalicylate_resistance'].apply(map_resistance)
result['IR_RIFABUTIN'] = df['rifabutin_resistance'].apply(map_resistance)
result['IR_SM'] = df['streptomycin_resistance'].apply(map_resistance)

result['IR_OTHER_DRUG'] = df['linezolid_resistance'].apply(lambda x: 'Linezolid' if pd.notna(x) else None)

result.head(10)

,RegisterCaseNumber,DiagnosisDate,Survey_Year,IR_Gatifloxacin,IR_Levofloxacin,IR_Ciprofloxacin,IR_Clarithromycin,IR_Clofazamine,IR_Cycloserine,IR_Pyridoxine,...,IR_CAPREOMYCIN,IR_ETHIONAMIDE,IR_KANAMYCIN,IR_MOXIFLOXACIN,IR_OFLOXACIN,IR_OTHER,IR_PAS,IR_RIFABUTIN,IR_SM,IR_OTHER_DRUG
0,1,2013-07-10,2013,,,,,,,,...,Resistant,Susceptible,Resistant,Resistant,Susceptible,Susceptible,Resistant,Resistant,Susceptible,Linezolid
1,2,2021-07-10,2021,,,,,,,,...,Susceptible,Resistant,Resistant,Resistant,Susceptible,Resistant,Resistant,Resistant,Susceptible,Linezolid
2,3,2017-07-25,2017,,,,,,,,...,Resistant,Susceptible,Resistant,Susceptible,Resistant,Resistant,Resistant,Resistant,Susceptible,Linezolid
3,4,2019-04-10,2019,,,,,,,,...,Resistant,Resistant,Resistant,Susceptible,Susceptible,Resistant,Susceptible,Resistant,Susceptible,Linezolid
4,5,2021-12-20,2021,,,,,,,,...,Susceptible,Susceptible,Susceptible,Susceptible,Resistant,Resistant,Resistant,Resistant,Susceptible,Linezolid
5,6,2015-02-27,2015,,,,,,,,...,Susceptible,Susceptible,Susceptible,Resistant,Susceptible,Susceptible,Resistant,Resistant,Resistant,Linezolid
6,7,2020-10-28,2020,,,,,,,,...,Susceptible,Resistant,Resistant,Susceptible,Resistant,Susceptible,Susceptible,Susceptible,Susceptible,Linezolid
7,8,2014-02-27,2014,,,,,,,,...,Susceptible,Resistant,Susceptible,Susceptible,Resistant,Susceptible,Susceptible,Susceptible,Susceptible,Linezolid
8,9,2017-11-28,2017,,,,,,,,...,Susceptible,Resistant,Susceptible,Susceptible,Susceptible,Susceptible,Resistant,Susceptible,Susceptible,Linezolid
9,10,2016-11-05,2016,,,,,,,,...,Susceptible,Susceptible,Susceptible,Resistant,Susceptible,Susceptible,Resistant,Resistant,Susceptible,Linezolid


##### Gender and origin

In [1564]:
SQL_QUERY = """
SELECT
    CASE
        WHEN gender LIKE 'Male' THEN 'M'
        WHEN gender LIKE 'Female' THEN 'F'
    END AS Gender,
    CASE
        WHEN origin = 'Canadian Born' THEN 8
        WHEN origin = 'Foreign Born' THEN 6
        ELSE 9
    END AS Origin
FROM df
"""
# sub_result = duckdb.sql(SQL_QUERY).to_df()
# result = pd.concat([result, sub_result], axis=1)

In [1565]:
result['Gender'] = df['gender'].map(lambda x: 'M' if x == 'Male' else 'F' if x == 'Female' else None)
result['Origin'] = df['origin'].map(lambda x: 8 if x == 'Canadian Born' else 6 if x == 'Foreign Born' else 9)

result.head(10)

,RegisterCaseNumber,DiagnosisDate,Survey_Year,IR_Gatifloxacin,IR_Levofloxacin,IR_Ciprofloxacin,IR_Clarithromycin,IR_Clofazamine,IR_Cycloserine,IR_Pyridoxine,...,IR_KANAMYCIN,IR_MOXIFLOXACIN,IR_OFLOXACIN,IR_OTHER,IR_PAS,IR_RIFABUTIN,IR_SM,IR_OTHER_DRUG,Gender,Origin
0,1,2013-07-10,2013,,,,,,,,...,Resistant,Resistant,Susceptible,Susceptible,Resistant,Resistant,Susceptible,Linezolid,F,6
1,2,2021-07-10,2021,,,,,,,,...,Resistant,Resistant,Susceptible,Resistant,Resistant,Resistant,Susceptible,Linezolid,F,9
2,3,2017-07-25,2017,,,,,,,,...,Resistant,Susceptible,Resistant,Resistant,Resistant,Resistant,Susceptible,Linezolid,None,9
3,4,2019-04-10,2019,,,,,,,,...,Resistant,Susceptible,Susceptible,Resistant,Susceptible,Resistant,Susceptible,Linezolid,M,9
4,5,2021-12-20,2021,,,,,,,,...,Susceptible,Susceptible,Resistant,Resistant,Resistant,Resistant,Susceptible,Linezolid,F,9
5,6,2015-02-27,2015,,,,,,,,...,Susceptible,Resistant,Susceptible,Susceptible,Resistant,Resistant,Resistant,Linezolid,F,8
6,7,2020-10-28,2020,,,,,,,,...,Resistant,Susceptible,Resistant,Susceptible,Susceptible,Susceptible,Susceptible,Linezolid,None,9
7,8,2014-02-27,2014,,,,,,,,...,Susceptible,Susceptible,Resistant,Susceptible,Susceptible,Susceptible,Susceptible,Linezolid,M,6
8,9,2017-11-28,2017,,,,,,,,...,Susceptible,Susceptible,Susceptible,Susceptible,Resistant,Susceptible,Susceptible,Linezolid,M,9
9,10,2016-11-05,2016,,,,,,,,...,Susceptible,Resistant,Susceptible,Susceptible,Resistant,Resistant,Susceptible,Linezolid,M,9


Another way to do this might be to ignore the cases that do not match the given conditions. Pandas populates these values with a NaN that can be filled using fillna. For example in the below code, if the field 'gender' in df does not match 'Male' or 'Female', then we get a NaN value in the result field. the `result.fillna('X')` fills it with 'X' instead [or None as in the original code]. Notice the row with index 2

In [1566]:
df['gender'].map({
    'Male': 'M',
    'Female': 'F'
})


0        F
1        F
2      NaN
3        M
4        F
      ... 
995    NaN
996      F
997      M
998    NaN
999    NaN
Name: gender, Length: 1000, dtype: object

In [1567]:
df['gender'].map({
    'Male': 'M',
    'Female': 'F'
}).fillna('X')

0      F
1      F
2      X
3      M
4      F
      ..
995    X
996    F
997    M
998    X
999    X
Name: gender, Length: 1000, dtype: object

##### Immigration fields

In [1568]:
SQL_QUERY = """
SELECT 
    CASE
        WHEN origin = 'Canadian Born' THEN 'Canada'
        WHEN (origin = 'Foreign Born' AND country_of_birth_combined LIKE 'Taiwan') THEN 'Taiwan (Province Of China)'
        WHEN (origin = 'Foreign Born' AND country_of_birth_combined LIKE 'Iran, Islamic Republic of') THEN 'Iran'
        WHEN (origin = 'Foreign Born' AND country_of_birth_combined Like '%Korea%') THEN 'Rep. Korea (South Korea)'
        WHEN (origin = 'Foreign Born' AND country_of_birth_combined LIKE 'Macao') THEN 'China, Macao SAR'
        WHEN (origin = 'Foreign Born' AND country_of_birth_combined LIKE 'Syrian Arab Republic') THEN 'Syrian Arab Republic (Syria)'
        WHEN (origin = 'Foreign Born' AND country_of_birth_combined LIKE 'Hong Kong') THEN 'China, Hong Kong SAR'
        WHEN (origin = 'Foreign Born' AND country_of_birth_combined LIKE 'Tibet') THEN 'Tibet (China)'
        WHEN (origin = 'Foreign Born' AND country_of_birth_combined IS NULL) THEN 'Unknown'
        ELSE country_of_birth_combined
    END AS Country_Of_Birth,
    immigration_arrival_date_combine AS DateOfArrival,
    CASE
        WHEN origin = 'Foreign Born' AND immigration_status_combined IS NULL THEN 9
        WHEN immigration_status_combined = 'Foreign-born Canadian Citizen' THEN 1
        WHEN immigration_status_combined = 'Permanent Resident' THEN 1
        WHEN immigration_status_combined = 'Refugee Claimant' THEN 2
        WHEN immigration_status_combined = 'Student Permit' THEN 6
        WHEN immigration_status_combined = 'Visitor' THEN 7
        WHEN immigration_status_combined = 'Work Permit' THEN 5
        WHEN immigration_status_combined = 'Other Current Immigration Status' THEN 8
        WHEN immigration_status_combined = 'Unknown' THEN 9
    END AS Immigration_Status,
    CASE
        WHEN immigration_status_combined = 'Other Current Immigration Status' THEN 'Not Specified'
        ELSE ''
    END AS OtherImmigStatus
FROM df
"""
# sub_result = duckdb.sql(SQL_QUERY).to_df()
# result = pd.concat([result, sub_result], axis=1)

In [1569]:
# Entire row can be passed to the method to use in apply function
def map_country_of_birth(row):
    if (row["origin"] == 'Canadian Born'): 
        return 'Canada'
    elif (row["origin"] == 'Foreign Born'):
        if(row['country_of_birth_combined'] ==  'Taiwan'): return 'Taiwan (Province Of China)'
        if(row['country_of_birth_combined'] ==  'Iran, Islamic Republic of'): return 'Iran'
        if('Korea' in row['country_of_birth_combined']): return 'Rep. Korea (South Korea)'
        if(row['country_of_birth_combined'] ==  'Macao'): return 'China, Macao SAR'
        if(row['country_of_birth_combined'] ==  'Syrian Arab Republic'): return 'Syrian Arab Republic (Syria)'
        if(row['country_of_birth_combined'] ==  'Hong Kong'): return 'China, Hong Kong SAR'
        if(row['country_of_birth_combined'] ==  'Tibet'): return 'Tibet (China)'
        if(row['country_of_birth_combined'] ==  'None'): return 'Unknown'
    return row['country_of_birth_combined']


def get_immigration_status(row):
    immigration_status_mapping = {
        'Foreign Born Canadian Citizen': 1,
        'Permanent Resident': 1,
        'Refugee Claimant': 2,
        'Student Permit': 6,
        'Visitor': 7,
        'Work Permit': 5,
        'Other Current Immigration Status': 8,
        'Unknown': 9,
    }
    if(row['origin'] == 'Foreign Born' and row['immigration_status_combined'] is None):
       return 9
    else:
        return immigration_status_mapping.get(row['immigration_status_combined'])
        
result['Country_Of_Birth'] = df.apply(map_country_of_birth, axis=1)
result['DateOfArrival'] = df['immigration_arrival_date_combine']
result['Immigration_Status'] = df.apply(get_immigration_status, axis=1)
result['OtherImmigStatus'] = df['immigration_status_combined'].map(lambda x: 'Not Specified' if x == 'Other Current Immigration Status' else '')
result.head(10)

,RegisterCaseNumber,DiagnosisDate,Survey_Year,IR_Gatifloxacin,IR_Levofloxacin,IR_Ciprofloxacin,IR_Clarithromycin,IR_Clofazamine,IR_Cycloserine,IR_Pyridoxine,...,IR_PAS,IR_RIFABUTIN,IR_SM,IR_OTHER_DRUG,Gender,Origin,Country_Of_Birth,DateOfArrival,Immigration_Status,OtherImmigStatus
0,1,2013-07-10,2013,,,,,,,,...,Resistant,Resistant,Susceptible,Linezolid,F,6,UK,1995-04-14,NaN,
1,2,2021-07-10,2021,,,,,,,,...,Resistant,Resistant,Susceptible,Linezolid,F,9,Canada,NaT,NaN,
2,3,2017-07-25,2017,,,,,,,,...,Resistant,Resistant,Susceptible,Linezolid,None,9,Canada,NaT,NaN,
3,4,2019-04-10,2019,,,,,,,,...,Susceptible,Resistant,Susceptible,Linezolid,M,9,Canada,NaT,NaN,
4,5,2021-12-20,2021,,,,,,,,...,Resistant,Resistant,Susceptible,Linezolid,F,9,Canada,NaT,NaN,
5,6,2015-02-27,2015,,,,,,,,...,Resistant,Resistant,Resistant,Linezolid,F,8,Canada,NaT,NaN,
6,7,2020-10-28,2020,,,,,,,,...,Susceptible,Susceptible,Susceptible,Linezolid,None,9,Canada,NaT,NaN,
7,8,2014-02-27,2014,,,,,,,,...,Susceptible,Susceptible,Susceptible,Linezolid,M,6,United States,2011-11-28,2.0,
8,9,2017-11-28,2017,,,,,,,,...,Resistant,Susceptible,Susceptible,Linezolid,M,9,Canada,NaT,NaN,
9,10,2016-11-05,2016,,,,,,,,...,Resistant,Resistant,Susceptible,Linezolid,M,9,Korea,1962-02-22,NaN,


##### Case detection fields

In [1570]:
SQL_QUERY="""
SELECT
    CASE
        WHEN method_of_detection = 'Symptoms compatible with site of disease' THEN 1
        WHEN method_of_detection = 'Incidental finding' THEN 2
        WHEN method_of_detection = 'Post-mortem' THEN 3
        WHEN method_of_detection = 'Contact investigation' THEN 4
        WHEN method_of_detection = 'Immigration medical surveillance' THEN 5
        WHEN method_of_detection = 'Other Screening' THEN 8
        WHEN method_of_detection = 'Other' THEN 9
        WHEN method_of_detection = 'Unknown' THEN 10
        WHEN origin = 'Foreign Born' AND method_of_detection IS NULL THEN 10
    END AS CaseFinding,
    CASE
        WHEN method_of_detection = 'Other' THEN 'Not specified'
        ELSE ''
    END AS CaseFindingOther,
    tb_body_site_category_phac AS Case_Comment
from df
"""
# sub_result = duckdb.sql(SQL_QUERY).to_df()
# result = pd.concat([result, sub_result], axis=1)

In [1571]:
def calculate_case_finding(row):
    method_of_detection = row['method_of_detection']
    if method_of_detection == 'Symptoms compatible with site of disease':
        return 1
    elif method_of_detection == 'Incidental finding':
        return 2
    elif method_of_detection == 'Post-mortem':
        return 3
    elif method_of_detection == 'Contact investigation':
        return 4
    elif method_of_detection == 'Immigration medical surveillance':
        return 5
    elif method_of_detection == 'Other Screening':
        return 8
    elif method_of_detection == 'Other':
        return 9
    elif method_of_detection == 'Unknown':
        return 10
    elif row['origin'] == 'Foreign Born' and pd.isna(method_of_detection):
        return 10
    else:
        return None

result['CaseFinding'] = df.apply(calculate_case_finding, axis=1)
result['CaseFindingOther'] = df['method_of_detection'].map(lambda x: 'Not specified' if x == 'Other' else '')
result['Case_Comment'] = df['tb_body_site_category_phac']
result.head(10)

,RegisterCaseNumber,DiagnosisDate,Survey_Year,IR_Gatifloxacin,IR_Levofloxacin,IR_Ciprofloxacin,IR_Clarithromycin,IR_Clofazamine,IR_Cycloserine,IR_Pyridoxine,...,IR_OTHER_DRUG,Gender,Origin,Country_Of_Birth,DateOfArrival,Immigration_Status,OtherImmigStatus,CaseFinding,CaseFindingOther,Case_Comment
0,1,2013-07-10,2013,,,,,,,,...,Linezolid,F,6,UK,1995-04-14,NaN,,9.0,Not specified,"Here's looking at you, kid."
1,2,2021-07-10,2021,,,,,,,,...,Linezolid,F,9,Canada,NaT,NaN,,2.0,,Ask not what your country can do for you; ask ...
2,3,2017-07-25,2017,,,,,,,,...,Linezolid,None,9,Canada,NaT,NaN,,3.0,,You're gonna need a bigger boat.
3,4,2019-04-10,2019,,,,,,,,...,Linezolid,M,9,Canada,NaT,NaN,,9.0,Not specified,Mama always said life was like a box of chocol...
4,5,2021-12-20,2021,,,,,,,,...,Linezolid,F,9,Canada,NaT,NaN,,NaN,,One morning I shot an elephant in my pajamas. ...
5,6,2015-02-27,2015,,,,,,,,...,Linezolid,F,8,Canada,NaT,NaN,,NaN,,I'm gonna make him an offer he can't refuse.
6,7,2020-10-28,2020,,,,,,,,...,Linezolid,None,9,Canada,NaT,NaN,,2.0,,The only thing we have to fear is fear itself.
7,8,2014-02-27,2014,,,,,,,,...,Linezolid,M,6,United States,2011-11-28,2.0,,NaN,,Ask not what your country can do for you; ask ...
8,9,2017-11-28,2017,,,,,,,,...,Linezolid,M,9,Canada,NaT,NaN,,9.0,Not specified,The only thing we have to fear is fear itself.
9,10,2016-11-05,2016,,,,,,,,...,Linezolid,M,9,Korea,1962-02-22,NaN,,9.0,Not specified,"Frankly, my dear, I don't give a damn."


In [1572]:
SQL_QUERY = """
SELECT 
    CASE
        WHEN previous_abnormal_chest_xray LIKE 'Yes' THEN 1
        WHEN previous_abnormal_chest_xray LIKE 'No' THEN 2
        ELSE 9
    END AS PrevAbnormalChestXRay,
    CASE
        WHEN tb_contact_within_2_years LIKE 'Yes' THEN 1
        WHEN tb_contact_within_2_years LIKE 'No' THEN 2
        ELSE 9
    END AS TBContact2Years,
    CASE
        WHEN diabetes_mellitus like '%Yes%' THEN 1
        WHEN diabetes_mellitus like '%No%' THEN 2
        ELSE 9
    END AS Diabetes,
    CASE
        WHEN kidney_disease_requiring_dialysi like '%Yes%' THEN 1
        WHEN kidney_disease_requiring_dialysi like '%No%' THEN 2
        ELSE 9
    END AS ChronicRenalFailure,
    CASE
        WHEN homelessness like '%Yes%' THEN 1
        WHEN homelessness like '%No%' THEN 2
        ELSE 9
    END AS Homeless,
    CASE
        WHEN longterm_corticosteriod_use like '%Yes%' THEN 1
        WHEN longterm_corticosteriod_use like '%No%' THEN 2
        ELSE 9
    END AS LTCorticosteroidUse,
    CASE
        WHEN injection_drug_use like '%Yes%' THEN 1
        WHEN injection_drug_use like '%No%' THEN 2
        ELSE 9
    END AS KnownSuspSubstanceAbuse,
    CASE
        WHEN solid_organ_transplant_candidate like '%Yes%' THEN 1
        WHEN solid_organ_transplant_candidate like '%No%' THEN 2
        ELSE 9
    END AS Transplant,
    CASE
        WHEN hiv_status IS NULL THEN 'Unknown'
        ELSE hiv_status
    END AS HIVStatus,
    CASE
        WHEN hiv_status LIKE 'Positive' OR hiv_status LIKE 'Negative' THEN '9999/99/99'
    END AS Date_Of_HIV_Test,
    '' AS LiveCorrectionSetting,
    '' AS TravelTBCountry,
    '' AS Howlong,
    CASE
        WHEN alcohol_use LIKE 'Yes' AND tobacco_use LIKE 'Yes' THEN 'alcohol and tobacco use'
        WHEN alcohol_use LIKE 'Yes' THEN 'alcohol use'
        WHEN tobacco_use LIKE 'Yes' THEN 'tobacco use'
    END AS OtherRiskFactors
FROM df
"""
# sub_result = duckdb.sql(SQL_QUERY).to_df()
# result = pd.concat([result, sub_result], axis=1)

In [1573]:
# In a dictionary we add data in the form of <key>: <value> pairs. For eg here, the column name in df is the key and the value it 
# becomes in result is the value
risk_factors = {
    'previous_abnormal_chest_xray': 'PrevAbnormalChestXRay',
    'tb_contact_within_2_years': 'TBContact2Years',
    'diabetes_mellitus': 'Diabetes',
    'kidney_disease_requiring_dialysi': 'ChronicRenalFailure',
    'homelessness': 'Homeless',
    'longterm_corticosteriod_use': 'LTCorticosteroidUse',
    'injection_drug_use': 'KnownSuspSubstanceAbuse',
    'solid_organ_transplant_candidate': 'Transplant'
}

def map_risk_factors(risk_factor):
    if pd.isna(risk_factor):
        return 9
    if "Yes" in risk_factor:
        return 1
    if "No" in risk_factor:
        return 2
    return 9

new_dict = {}
# We can iterate over dictionaries, ie, go through each key, value pair one by one. We create a new dictionary, convert that to a dataframe 
# and then concatenate it to the original dataframe for performance reasons. It doesn't really matter in this use case but we would see warnings
# if new columns are created continuously in dataframes.
for key, value in risk_factors.items():
    new_dict[value] = df[key].map(map_risk_factors)

# A new dataframe can be created from a dictionary as well. 
result = pd.concat([result, pd.DataFrame(new_dict)], axis=1)

result['HIVStatus'] = df['hiv_status'].fillna('Unknown')

result.head(10)

,RegisterCaseNumber,DiagnosisDate,Survey_Year,IR_Gatifloxacin,IR_Levofloxacin,IR_Ciprofloxacin,IR_Clarithromycin,IR_Clofazamine,IR_Cycloserine,IR_Pyridoxine,...,Case_Comment,PrevAbnormalChestXRay,TBContact2Years,Diabetes,ChronicRenalFailure,Homeless,LTCorticosteroidUse,KnownSuspSubstanceAbuse,Transplant,HIVStatus
0,1,2013-07-10,2013,,,,,,,,...,"Here's looking at you, kid.",2,2,1,1,9,1,9,9,Unknown
1,2,2021-07-10,2021,,,,,,,,...,Ask not what your country can do for you; ask ...,9,1,1,9,2,2,9,2,Unknown
2,3,2017-07-25,2017,,,,,,,,...,You're gonna need a bigger boat.,2,1,1,2,9,1,9,9,Test refused
3,4,2019-04-10,2019,,,,,,,,...,Mama always said life was like a box of chocol...,9,2,9,9,2,1,9,1,Unknown
4,5,2021-12-20,2021,,,,,,,,...,One morning I shot an elephant in my pajamas. ...,1,9,1,2,2,1,1,9,Unknown
5,6,2015-02-27,2015,,,,,,,,...,I'm gonna make him an offer he can't refuse.,1,2,2,1,9,9,2,1,Unknown
6,7,2020-10-28,2020,,,,,,,,...,The only thing we have to fear is fear itself.,9,9,1,1,9,9,9,9,Test not offered
7,8,2014-02-27,2014,,,,,,,,...,Ask not what your country can do for you; ask ...,9,1,9,9,9,9,9,1,Test refused
8,9,2017-11-28,2017,,,,,,,,...,The only thing we have to fear is fear itself.,2,1,9,2,9,1,9,2,Positive
9,10,2016-11-05,2016,,,,,,,,...,"Frankly, my dear, I don't give a damn.",1,9,2,9,9,9,1,9,Negative


#### 2.2. Data Conversions:  
  
To maintain consistency of data from the different provinces, there is a necessity to convert certain fields from one data type to another. For instance, reporting is easier if age is a numeric class and so we convert the ages to a numeric class that denote age groups. 

#### 2.3. Adding new variables:

There is also a necessity to add new columns for data uniformity for reporting. One example for this would be introducing a new variable `Subject_Code` that is formed by appending the `Survey_Year` and the `RegisterCaseNumber` if the Survey_Year is not null, for the province of British Columbia.  
  
The SQL query for such these operations and the code for executing the query would look like the follows:


In [1574]:
SQL_QUERY = """SELECT 
    *, 
    CASE 
        WHEN AGE >= -1 AND AGE < 1 THEN 1 
        WHEN AGE >= 1 AND AGE < 5 THEN 2 
        WHEN AGE >= 5 AND AGE < 15 THEN 3 
        WHEN AGE >= 15 AND AGE < 25 THEN 4 
        WHEN AGE >= 25 AND AGE < 35 THEN 5 
        WHEN AGE >= 35 AND AGE < 45 THEN 6 
        WHEN AGE >= 45 AND AGE < 55 THEN 7 
        WHEN AGE >= 55 AND AGE < 65 THEN 8 
        WHEN AGE >= 65 AND AGE < 75 THEN 9 
        WHEN AGE >= 75 AND AGE < 120 THEN 10 
        ELSE 99 
    END AS AgeGroup, 
    CASE 
        WHEN survey_year IS NOT NULL THEN survey_year || '-' || RegisterCaseNumber 
    END AS Subject_Code 
FROM result; 
"""

# result = duckdb.sql(SQL_QUERY).to_df()
# result[['Age', 'AgeGroup', 'Survey_Year', 'RegisterCaseNumber', 'Subject_Code']].head(5)

The below code has the SQL query executed using the Pandas API to segment the row to various buckets of age groups using the [cut](https://pandas.pydata.org/docs/reference/api/pandas.cut.html) method. The arguments to cut used are:

data: The data to be segmented - the age column.
bins: The edges that determine the boundary of each bin. Here [-1, 1, 5, 15, 25, 35, 45, 55, 65, 75, 120] means the bins are: (-1 to <1, 1 to <5 and so on).
labels: The different categories to identify.

The length of labels will be one less than the length of bins because bins are used to indicate the edges of each group that we divide into.
So in this example, label 1 is for group -1 to 1 (1 not included), label 2 is for 1 to 5 (5 not included) and so on. Also, the upper numbers are not included because we pass 'False' to the argument right and the left edges are included as we pass True to include_lowest. This is done to be similar to the query above.

In [1575]:
result['AgeGroup'] = pd.cut(result['Age'],
    bins=[-1, 1, 5, 15, 25, 35, 45, 55, 65, 75, 120],
    labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    include_lowest=True,
    right=False
)
result['Subject_Code'] = result.apply(
    lambda row: f"{row['Survey_Year']}-{row['RegisterCaseNumber']}" if not pd.isnull(row['Survey_Year']) else None,
    axis=1
)

result.head(10)

,RegisterCaseNumber,DiagnosisDate,Survey_Year,IR_Gatifloxacin,IR_Levofloxacin,IR_Ciprofloxacin,IR_Clarithromycin,IR_Clofazamine,IR_Cycloserine,IR_Pyridoxine,...,TBContact2Years,Diabetes,ChronicRenalFailure,Homeless,LTCorticosteroidUse,KnownSuspSubstanceAbuse,Transplant,HIVStatus,AgeGroup,Subject_Code
0,1,2013-07-10,2013,,,,,,,,...,2,1,1,9,1,9,9,Unknown,8,2013-1
1,2,2021-07-10,2021,,,,,,,,...,1,1,9,2,2,9,2,Unknown,9,2021-2
2,3,2017-07-25,2017,,,,,,,,...,1,1,2,9,1,9,9,Test refused,4,2017-3
3,4,2019-04-10,2019,,,,,,,,...,2,9,9,2,1,9,1,Unknown,5,2019-4
4,5,2021-12-20,2021,,,,,,,,...,9,1,2,2,1,1,9,Unknown,8,2021-5
5,6,2015-02-27,2015,,,,,,,,...,2,2,1,9,9,2,1,Unknown,9,2015-6
6,7,2020-10-28,2020,,,,,,,,...,9,1,1,9,9,9,9,Test not offered,9,2020-7
7,8,2014-02-27,2014,,,,,,,,...,1,9,9,9,9,9,1,Test refused,9,2014-8
8,9,2017-11-28,2017,,,,,,,,...,1,9,2,9,1,9,2,Positive,4,2017-9
9,10,2016-11-05,2016,,,,,,,,...,9,2,9,9,9,1,9,Negative,6,2016-10


#### 2.4. Preparing Data for Upload  
  
 To prepare the final data for upload to the storage system in a standardized format, follow these steps:

In [1576]:
SQL_QUERY = """SELECT
    RegisterCaseNumber,
    Subject_Code,
    'British Columbia' AS ReportingProvince,
    '9999/99/99' AS Date_Of_Birth,
    Gender,
    Age,
    AgeGroup,
    '' AS City,
    '' AS Health_Unit,
    'Z9Z' AS PostalCode,
    Origin,
    '' AS Place_Lived,
    '' AS Other_Aboriginal,
    '' AS COBMother,
    '' AS COBFather,
    Country_Of_Birth,
    DateOfArrival,
    Immigration_Status,
    OtherImmigStatus,
    'Active' AS Subject_Status,
    Survey_Year,
    DiagnosisDate,
    '' AS ICDOption,
    '' AS ICD9,
    '' AS ICD10,
    '' AS Diagnostics_Site,
    '' AS ChestXRay,
    '' AS IfAbnormal,
    '' AS M_Sputum,
    '' AS M_BronchialWash,
    '' AS M_GIWash,
    '' AS M_NodeBiopsy,
    '' AS M_Urine,
    '' AS M_CSF,
    '' AS M_Other,
    '' AS C_Sputum,
    '' AS C_BronchialWash,
    '' AS C_GIWash,
    '' AS C_NodeBiopsy,
    '' AS C_Urine,
    '' AS C_CSF,
    '' AS C_Other,
    '' AS BS_Microscopy,
    '' AS BS_Culture,
    CaseCriteria,
    Initial_Resistance,
    IR_INH,
    IR_EMB,
    IR_RMP,
    IR_PZA,
    IR_Amikacin,
    IR_Capreomycin,
    IR_Ciprofloxacin,
    IR_Clarithromycin,
    IR_Clofazamine,
    IR_Cycloserine,
    IR_Ethionamide,
    IR_Gatifloxacin,
    IR_Kanamycin,
    IR_Levofloxacin,
    IR_Moxifloxacin,
    IR_Ofloxacin,
    IR_Other,
    IR_PAS,
    IR_Pyridoxine,
    IR_Rifabutin,
    IR_Rifapentine,
    IR_SM,
    IR_Other_Drug,
    '' AS GenotypingResults,
    '' AS GenotypingSpoligotyping,
    '' AS MIRU,
    NULL AS RFLP,
    '' AS DateTreatmentStarted,
    '' AS DP_INH,
    '' AS DP_EMB,
    '' AS DP_RMP,
    '' AS DP_PZA,
    NULL AS DP_Amikacin,
    NULL AS DP_Capreomycin,
    NULL AS DP_Ciprofloxacin,
    NULL AS DP_Clarithromycin,
    NULL AS DP_Clofazamine,
    NULL AS DP_Cycloserine,
    NULL AS DP_Ethionamide,
    NULL AS DP_Gatifloxacin,
    NULL AS DP_Kanamycin,
    NULL AS DP_Levofloxacin,
    NULL AS DP_Moxifloxacin,
    NULL AS DP_NoDrugsPrescribed,
    NULL AS DP_Ofloxacin,
    '1' AS DP_Other,
    NULL AS DP_PAS,
    NULL AS DP_Pyridoxine,
    NULL AS DP_Rifabutin,
    NULL AS DP_Rifapentine,
    NULL AS DP_SM,
    NULL AS DP_Unknown,
    'Not Specified' AS DP_Other_Drug,
    CaseFinding,
    CaseFindingOther,
    '' AS patientDiedBeforeDuring,
    '' AS DidPatientDie,
    '' AS DateOfDeath,
    '' AS CauseOfDeath,
    '' AS FirstEpisodeOfTB,
    '' AS PreviousDiagnosisYear,
    '' AS PreviousDiagCountry,
    NULL AS FE_EMB,
    NULL AS FE_INH,
    NULL AS FE_Other,
    NULL AS FE_PZA,
    NULL AS FE_RMP,
    NULL AS FE_Rifapentine,
    NULL AS FE_SM,
    '' AS FE_Unknown,
    '' AS PreviousTreatmentCompleted,
    '' AS LastDatePreviousTreatment,
    '' AS OtherFirstEpisodeTreatment,
    HIVStatus,
    Date_Of_HIV_Test,
    KnownSuspSubstanceAbuse,
    Transplant,
    ChronicRenalFailure,
    TBcontact2Years,
    PrevAbnormalChestXRay,
    Diabetes,
    LTCorticosteroidUse,
    LiveCorrectionSetting,
    Homeless,
    TravelTBCountry,
    OtherRiskFactors,
    HowLong,
    Case_Comment
FROM result;
"""

# result = duckdb.sql(SQL_QUERY).to_df()
# result.head(5)

In [1577]:
# Only the additional fields are needed to be defined.

new_columns = pd.DataFrame({
    'ReportingProvince':'British Columbia',
    'City': '',
    'Health_Unit': '',
    'PostalCode': 'Z9Z',
    'Place_Lived': '',
    'Other_Aboriginal': '',
    'COBMother': '',
    'COBFather': '',
    'ICDOption': '',
    'ICD9': '',
    'ICD10': '',
    'Diagnostics_Site': '',
    'ChestXRay': '',
    'IfAbnormal': '',
    'M_Sputum': '',
    'M_BronchialWash': '',
    'M_GIWash': '',
    'M_NodeBiopsy': '',
    'M_Urine': '',
    'M_CSF': '',
    'M_Other': '',
    'C_Sputum': '',
    'C_BronchialWash': '',
    'C_GIWash': '',
    'C_NodeBiopsy': '',
    'C_Urine': '',
    'C_CSF': '',
    'C_Other': '',
    'BS_Microscopy': '',
    'BS_Culture': '',
    'GenotypingResults': '',
    'GenotypingSpoligotyping': '',
    'MIRU': '',
    'RFLP': None,
    'DateTreatmentStarted': '',
    'DP_INH': '',
    'DP_EMB': '',
    'DP_RMP': '',
    'DP_PZA': '',
    'DP_Amikacin': None,
    'DP_Capreomycin': None,
    'DP_Ciprofloxacin': None,
    'DP_Clarithromycin': None,
    'DP_Clofazamine': None,
    'DP_Cycloserine': None,
    'DP_Ethionamide': None,
    'DP_Gatifloxacin': None,
    'DP_Kanamycin': None,
    'DP_Levofloxacin': None,
    'DP_Moxifloxacin': None,
    'DP_NoDrugsPrescribed': None,
    'DP_Ofloxacin': None,
    'DP_Other': '1',
    'DP_PAS': None,
    'DP_Pyridoxine': None,
    'DP_Rifabutin': None,
    'DP_Rifapentine': None,
    'DP_SM': None,
    'DP_Unknown': None,
    'DP_Other_Drug': 'Not Specified',
    'patientDiedBeforeDuring': '',
    'DidPatientDie': '',
    'DateOfDeath': '',
    'CauseOfDeath': '',
    'FirstEpisodeOfTB': '',
    'PreviousDiagnosisYear': '',
    'PreviousDiagCountry': '',
    'FE_Unknown': '',
    'PreviousTreatmentCompleted': '',
    'LastDatePreviousTreatment': '',
    'OtherFirstEpisodeTreatment': ''
}, index=range(len(result)))

# Add the new columns to the 'result' DataFrame
result = pd.concat([result, new_columns], axis=1)

In [1587]:
new_columns

,City,Health_Unit,PostalCode,Place_Lived,Other_Aboriginal,COBMother,COBFather,ICDOption,ICD9,ICD10,...,DidPatientDie,DateOfDeath,CauseOfDeath,FirstEpisodeOfTB,PreviousDiagnosisYear,PreviousDiagCountry,FE_Unknown,PreviousTreatmentCompleted,LastDatePreviousTreatment,OtherFirstEpisodeTreatment
0,,,Z9Z,,,,,,,,...,,,,,,,,,,
1,,,Z9Z,,,,,,,,...,,,,,,,,,,
2,,,Z9Z,,,,,,,,...,,,,,,,,,,
3,,,Z9Z,,,,,,,,...,,,,,,,,,,
4,,,Z9Z,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,,,Z9Z,,,,,,,,...,,,,,,,,,,
996,,,Z9Z,,,,,,,,...,,,,,,,,,,
997,,,Z9Z,,,,,,,,...,,,,,,,,,,
998,,,Z9Z,,,,,,,,...,,,,,,,,,,


In [1578]:
result.head(10)

,RegisterCaseNumber,DiagnosisDate,Survey_Year,IR_Gatifloxacin,IR_Levofloxacin,IR_Ciprofloxacin,IR_Clarithromycin,IR_Clofazamine,IR_Cycloserine,IR_Pyridoxine,...,DidPatientDie,DateOfDeath,CauseOfDeath,FirstEpisodeOfTB,PreviousDiagnosisYear,PreviousDiagCountry,FE_Unknown,PreviousTreatmentCompleted,LastDatePreviousTreatment,OtherFirstEpisodeTreatment
0,1,2013-07-10,2013,,,,,,,,...,,,,,,,,,,
1,2,2021-07-10,2021,,,,,,,,...,,,,,,,,,,
2,3,2017-07-25,2017,,,,,,,,...,,,,,,,,,,
3,4,2019-04-10,2019,,,,,,,,...,,,,,,,,,,
4,5,2021-12-20,2021,,,,,,,,...,,,,,,,,,,
5,6,2015-02-27,2015,,,,,,,,...,,,,,,,,,,
6,7,2020-10-28,2020,,,,,,,,...,,,,,,,,,,
7,8,2014-02-27,2014,,,,,,,,...,,,,,,,,,,
8,9,2017-11-28,2017,,,,,,,,...,,,,,,,,,,
9,10,2016-11-05,2016,,,,,,,,...,,,,,,,,,,


### Step 3. Storing Data Files

[Parquet](https://www.databricks.com/glossary/what-is-parquet#:~:text=What%20is%20Parquet%3F,handle%20complex%20data%20in%20bulk.) is a columnar data storage file format that is specifically designed to be efficient for data processing. It stores data in a column format that helps better compression and faster access. Typically, parquet format also include metadata that contains the schema definition of the data.   
  
Another advantage with parquet is that it allows schema evolution - columns can be added or removed. This can be particularly useful for accommodating the changes to the structure of the data in this project – for example, to accommodate a change in the case definition of Tuberculosis. 

When storing the data, partitioning columns can be set that indicate the directory structure. Pandas supports writing as parquet with [`to_parquet`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_parquet.html) method where we indicate with arguments parameters like:  

`path`: Directory path to store files  
`partition_cols`: The columns used to partition the data. Subdirectories are created for faster access when querying these columns. This can be set if certain columns are frequently used for reporting queries such as `Survey_Year`.



In [1579]:
result.to_parquet("tests/CaseFileOutput/", partition_cols=['Survey_Year'])

/Users/Shared/miniconda3/envs/tb-analytics/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


## Outcome Files

The process for Outcome files is pretty much the same as above using similar steps and operations.

In [1580]:
DATA_FILE=Path(".", "tests", "BCOutcomesUploading.xlsx")
df = pd.read_excel(DATA_FILE)

In [1581]:
OUTCOME_PRELIM_QUERY = """
select
          classification_year as outcome_surveyyear,
          client_id_phac as Re_RegisterCaseNume_2, /*Subhect_Code? */
          classification_date as DiagnosisDate_2 ,
          treatment_start_date as InitTreatDate_2 ,
          treatment_end_date as LastDayOfTreatment_2,
          death_date_combined as DateOfDeath_2 ,
          '' as MajorTreatmentMode_2_1,
          '' as ComplianceEstimate_2_1,
          '' as TRT_OTHER_1
    from df;
"""
result = duckdb.sql(OUTCOME_PRELIM_QUERY).to_df()

In [1582]:
SQL_QUERY = """
    select
          case
               when reason_treatment_ended_combined like 'Adherent' then 2
               when reason_treatment_ended_combined like 'Deceased' then 3
               when reason_treatment_ended_combined is null and death_date_combined is not null then 3
               when reason_treatment_ended_combined like 'Other' and death_date_combined is not null then 3
               when reason_treatment_ended_combined like 'Drug reaction/intolerance' then 10
               when reason_treatment_ended_combined like 'Left BC-within Canada' then 4
               when reason_treatment_ended_combined like 'Left Province' then 4
               when reason_treatment_ended_combined like 'Transferred' then 4
               when reason_treatment_ended_combined like 'Left Canada' then 6
               when reason_treatment_ended_combined like 'Lost to follow up' then 6 /*3*/ /* 7 for category 6 */
               when reason_treatment_ended_combined like 'Non-adherent' then 8 /*1*/
               when reason_treatment_ended_combined like 'Other' then 8
               when reason_treatment_ended_combined like 'No Data' then 9
               else 9 /*40-21*/
               end as TreatmentOutcome_2,
          case
               when reason_treatment_ended_combined like 'Non-adherent' then 'non-compliant'
               when reason_treatment_ended_combined like 'Other' then 'Unknown'
               else ''
               end AS OtherTreatmentOutcome_2
    from df;
"""
sub_result = duckdb.sql(SQL_QUERY).to_df()
result = pd.concat([result, sub_result], axis=1)

In [1583]:
SQL_QUERY = """
select
          case
               when cause_of_death_combined like 'Contributed, but wasn''t the underlying cause'
                    then 'TB contributed to death but was not the underlying cause'
               when cause_of_death_combined like 'Did not contribute to death/incidental'
                    then 'TB did not contribute to death'
               when cause_of_death_combined like 'Underlying cause of death'
                    then 'TB was the cause of death'
               when cause_of_death_combined like 'Unknown'
                    then 'Unknown'
               end as CauseOfDeath_2,
          case
               when reason_treatment_ended_combined like 'Left BC-within Canada'
                    or reason_treatment_ended_combined like 'Left Province'
                    or reason_treatment_ended_combined like 'Transferred'
                    /*transferred under diff variable - if_transfer in CTBRS */
                    or reason_treatment_ended_combined like 'Left Canada' then 'Unknown'
               else ''
               end as NewJurisdiction_2
    from df;
"""
sub_result = duckdb.sql(SQL_QUERY).to_df()
result = pd.concat([result, sub_result], axis=1)

In [1584]:
SQL_QUERY = """
    select
        case
               when MajorTreatmentMode_2_1 is null then 4
            end 
        as MajorTreatmentMode_2,
        '' as OtherMode,
        case
               when ComplianceEstimate_2_1 is null then 'Unknown'
               end 
        as ComplianceEstimate_2
    from result;
"""
sub_result = duckdb.sql(SQL_QUERY).to_df()
outcome_prelim = pd.concat([result, sub_result], axis=1)

In [1585]:
FINAL_QUERY = """
select
          outcome_surveyyear,
          '' as TBPC_NUMBER,
          DiagnosisDate_2,
          '' as IfTransfer_2,
          Re_RegisterCaseNume_2,
          '' as Re_UniqueID_2,
          '' as NoTreatmentRecvd,
          InitTreatDate_2,
          LastDayOfTreatment_2,
          '' as ACQUIRED_RESISTANCE,
          '' as AR_INH,
          '' as AR_EMB,
          '' as AR_RMP,
          '' as AR_PZA,
          '' as AR_Amikacin,
          '' as AR_Capreomycin,
          '' as AR_Ciprofloxacin,
          '' as AR_Clarithromycin,
          '' as AR_Clofazamine,
          '' as AR_Cycloserine,
          '' as AR_Ethionamide,
          '' as AR_Gatifloxacin,
          '' as AR_Kanamycin,
          '' as AR_Levofloxacin,
          '' as AR_Moxifloxacin,
          '' as AR_Ofloxacin,
          '' as AR_Other,
          '' as AR_PAS,
          '' as AR_Pyridoxine,
          '' as AR_Rifabutin,
          '' as AR_Rifapentine,
          '' as AR_SM,
          '' as AR_Other_Drug,
          '' as TRT_INH,
          '' as TRT_EMB,
          '' as TRT_RMP,
          '' as TRT_PZA,
          '' as TRT_AMI,
          '' as TRT_CAP,
          '' as TRT_CIP,
          '' as TRT_CLA,
          '' as TRT_CLO,
          '' as TRT_CYC,
          '' as TRT_ETH,
          '' as TRT_GAT,
          '' as TRT_KAN,
          '' as TRT_LEV,
          '' as TRT_MOX,
          '' as TRT_OFL,
          case
               when TRT_OTHER_1 is null then 1
               end as TRT_OTHER,
          '' as TRT_PAS,
          '' as TRT_PRY,
          '' as TRT_RIFA,
          '' as TRT_RIFAPENT,
          '' as TRT_SM,
          MajorTreatmentMode_2,
          OtherMode,
          ComplianceEstimate_2,
          TreatmentOutcome_2,
          OtherTreatmentOutcome_2,
          NewJurisdiction_2,
          CauseOfDeath_2,
          DateOfDeath_2,
          '' as Comment_2,
          '' as Close_ContactEvaluated,
          '' as Close_ActiveTB,
          '' as Close_LTBI,
          '' as Close_BeginningTrt,
          '' as Close_CompletingTrt,
          '' as Casual_ContactEvaluated,
          '' as Casual_ActiveTB,
          '' as Casual_LTBI,
          '' as Casual_BeginningTrt,
          '' as Casual_CompletingTrt,
          '' as Community_ContactEvaluated,
          '' as Community_ActiveTB,
          '' as Community_LTBI,
          '' as Community_BeginningTrt,
          '' as Community_CompletingTrt
    from outcome_prelim;
"""
result_outcome = duckdb.sql(FINAL_QUERY).to_df()
result_outcome.head(10)

,outcome_surveyyear,TBPC_NUMBER,DiagnosisDate_2,IfTransfer_2,Re_RegisterCaseNume_2,Re_UniqueID_2,NoTreatmentRecvd,InitTreatDate_2,LastDayOfTreatment_2,ACQUIRED_RESISTANCE,...,Casual_ContactEvaluated,Casual_ActiveTB,Casual_LTBI,Casual_BeginningTrt,Casual_CompletingTrt,Community_ContactEvaluated,Community_ActiveTB,Community_LTBI,Community_BeginningTrt,Community_CompletingTrt
0,2021,,2021-09-12,,1,,,1993-12-24,2005-01-30,,...,,,,,,,,,,
1,2015,,2015-01-08,,2,,,1996-04-05,2015-11-19,,...,,,,,,,,,,
2,2022,,2022-05-22,,3,,,1994-12-22,2018-11-26,,...,,,,,,,,,,
3,2015,,2015-01-04,,4,,,1997-02-08,2018-11-29,,...,,,,,,,,,,
4,2019,,2019-02-03,,5,,,1996-06-10,1999-01-25,,...,,,,,,,,,,
5,2023,,2023-11-20,,6,,,1997-01-13,2013-10-28,,...,,,,,,,,,,
6,2016,,2016-11-21,,7,,,2002-07-22,2016-09-12,,...,,,,,,,,,,
7,2018,,2018-04-24,,8,,,1995-12-23,2022-04-25,,...,,,,,,,,,,
8,2018,,2018-11-30,,9,,,1998-06-05,2012-05-31,,...,,,,,,,,,,
9,2022,,2022-02-25,,10,,,1999-03-28,2001-05-01,,...,,,,,,,,,,


In [1586]:
result_outcome.to_parquet("tests/OutcomeFileOutput/", partition_cols=['outcome_surveyyear'])

/Users/Shared/miniconda3/envs/tb-analytics/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
